In [2]:
#import dependencies
import pandas as pd, gmaps, requests

#import api K
from config import g_key

In [4]:
#import csv to df
city_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
#temp_shift = city_df["Max Temp"].min()
#city_df["Max Temp"] = city_df["Max Temp"].add(-temp_shift,fill_value=0)
#city_df.loc[city_df["Max Temp"] < 0, "Max Temp"] = 0
city_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desc,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,0,Puerto Ayora,EC,2020-02-23 17:58:38,-0.74,-90.35,84.20,74,20,12.75,few clouds,0.00,0.0
1,1,Rikitea,PF,2020-02-23 17:58:23,-23.12,-134.97,78.37,74,2,8.25,light rain,0.44,0.0
2,2,Provideniya,RU,2020-02-23 17:58:49,64.38,-173.30,-9.76,73,0,2.24,clear sky,0.00,0.0
3,3,Lukovetskiy,RU,2020-02-23 18:02:54,64.30,41.92,35.60,80,90,11.18,overcast clouds,0.00,0.0
4,4,Cape Town,ZA,2020-02-23 17:58:43,-33.93,18.42,75.20,44,0,21.92,clear sky,0.00,0.0


In [5]:
#ask customer to add min and max temp values, and precipitation
min_temp = float(input("What is the min temp you would like?"))
max_temp = float(input("What is the max temp you would like?"))
raining = str(input("Do you want it to be raining? (yes/no)"))
snowing = str(input("Do you want it to be snowing? (yes/no)"))

What is the min temp you would like?-5
What is the max temp you would like?50
Do you want it to be raining? (yes/no)no
Do you want it to be snowing? (yes/no)yes


In [6]:
#create new df based on customer temp preferences and raining or snowing
preferred_cities_df = city_df.loc[(city_df["Max Temp"] >= min_temp) & 
                          (city_df["Max Temp"] <= max_temp)]

if raining.lower() == "yes":
    preferred_cities_df  = preferred_cities_df.loc[
        (preferred_cities_df["Rain Inches (last 3 hrs)"]>0)]
else:
    preferred_cities_df  = preferred_cities_df.loc[
        (preferred_cities_df["Rain Inches (last 3 hrs)"]==0)]

if snowing.lower() == "yes":
    preferred_cities_df  = preferred_cities_df.loc[
        (preferred_cities_df["Snow Inches (last 3 hrs)"]>0)]
else:
    preferred_cities_df  = preferred_cities_df.loc[
        (preferred_cities_df["Snow Inches (last 3 hrs)"]==0)]

preferred_cities_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desc,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
21,21,Dikson,RU,2020-02-23 17:58:46,73.51,80.55,30.27,95,100,27.36,snow,0.0,1.75
33,33,Severo-Kurilsk,RU,2020-02-23 18:03:02,50.68,156.12,23.50,96,100,37.63,snow,0.0,3.06
75,75,Karpogory,RU,2020-02-23 18:03:12,64.00,44.45,33.67,89,100,15.17,light snow,0.0,0.19
97,97,Khatanga,RU,2020-02-23 18:03:20,71.97,102.50,12.67,92,98,15.46,light snow,0.0,0.25
111,111,Omagari,JP,2020-02-23 18:03:23,39.45,140.48,34.00,65,93,5.01,light snow,0.0,0.13


In [11]:
#create hotel_df with less columns and new key
hotel_df = preferred_cities_df.copy()
hotel_df = hotel_df[["City","Country","Max Temp","Current Desc", "Lat","Lng",
                    "Rain Inches (last 3 hrs)", "Snow Inches (last 3 hrs)"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Desc,Lat,Lng,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs),Hotel Name
21,Dikson,RU,30.27,snow,73.51,80.55,0.0,1.75,
33,Severo-Kurilsk,RU,23.50,snow,50.68,156.12,0.0,3.06,
75,Karpogory,RU,33.67,light snow,64.00,44.45,0.0,0.19,
97,Khatanga,RU,12.67,light snow,71.97,102.50,0.0,0.25,
111,Omagari,JP,34.00,light snow,39.45,140.48,0.0,0.13,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 50000,
    "type": "lodging",
    "key": g_key
}

#iterate through hotel df
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    try:
        # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

Hotel not found... skipping


In [13]:
#set info box per Gmaps
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Desc} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [16]:
#configure gmaps to use API
gmaps.configure(api_key=g_key)

In [21]:
# Heatmap of temperature for vacation spots
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
#create output file
output_data_file = "weather_data/WeatherPy_vacation.csv"
#export to file
hotel_df.to_csv(output_data_file,index_label="City_ID")

In [ ]:
Sernur
Georgiyevskoye
Kholm
Syasstroy